In [6]:
!pip install mtcnn

In [7]:
import sys, os
import numpy as np
from math import *
import math
from PIL import Image
from matplotlib import pyplot
from matplotlib.patches import Rectangle, Circle
import matplotlib.pyplot as plt
import cv2
import mtcnn
from mtcnn.mtcnn import MTCNN
import random

In [8]:
def bb_intersection_over_union(box1, box2):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(box1[0], box2[0])
	yA = max(box1[1], box2[1])
	xB = min(box1[2], box2[2])
	yB = min(box1[3], box2[3])
	# compute the area of intersection rectangle
	interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
	# compute the area of both the prediction and ground-truth
	# rectangles
	box1Area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
	box2Area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(box1Area + box2Area - interArea)
	# return the intersection over union value
	return iou

In [9]:
def calculate_iou(gtbox,MTCNNbox):
  no_counter = 0
  text = img_file + "\n" + str(num_faces) + "," + str(len(MTCNNbox)) + "\n"
  for boxA in gtbox:
    if len(MTCNNbox)==0:
      text += ','.join([str(elem) for elem in boxA]) + '/' + "None" + '/' + "0.0" + '/' + "0.0" + '\n'
      no_counter += 1
    else:
      test_iou = []
      for boxB in MTCNNbox:
        test_iou.append(bb_intersection_over_union(boxA, boxB[0]))
      #FN
      if max(test_iou) == 0.0:
        text += ','.join([str(elem) for elem in boxA]) + "/" + "None" + '/' + "0.0" + "/" + str(max(test_iou)) + "\n"
        no_counter += 1
      #TP
      else:
        for i,u in enumerate(test_iou):
          if u==max(test_iou):
            index=i
            text += ','.join([str(elem) for elem in boxA]) + "/" + ','.join([str(elem) for elem in MTCNNbox[i][0]]) + "/" + str(MTCNNbox[i][1]) + "/" + str(max(test_iou)) + "\n"
            break
    """#FP
    for boxC in mtcnnbox:
      test_iou=[]
      for boxD in gtbox:
        test_iou.append(bb_intersection_over_union(boxC, boxD))
        if max(test_iou)==0.0:
          text += 'None' + '/' + ','.join([str(elem) for elem in boxC])' + '/' + str(correct_iou) + '\n' """
  
  #print(text)
  return text

In [ ]:
folds = ["01","02","03","04","05","06","07","08","09","10"]
for fold_num in folds:

  rect_filename = '/content/drive/MyDrive/paper_face_recognitioin/FDDB-folds/rect/FDDB-fold-{f_name}-rectList.txt'.format(f_name = fold_num)
  iou_filename = '/content/drive/MyDrive/paper_face_recognitioin/FDDB-folds/iou/MTCNN/FDDB-fold-{f_name}-MTCNN-IOU.txt'.format(f_name = fold_num)

  with open(rect_filename) as f:
    lines = [line.rstrip('\n') for line in f]

  counter = 0
  f = open(iou_filename,'w')
  while counter < len(lines):
    img_file = lines[counter] 
    image = cv2.imread(img_file)
    num_faces = int(lines[counter+1])

    Confidence = []
    gtbox = []
    for j in range(num_faces):
      gt = [int(s) for s in lines[counter+j+2].split(',')]
      gtbox.append(gt)

    detector = MTCNN()
    faces = detector.detect_faces(image)
    MTCNNrect = []
    for face in faces:
      x, y, width, height = face['box']
      confidence = face["confidence"]
      Confidence.append(confidence)
      mtcnn = [x, y, x+width, y+height]
      MTCNNrect.append(mtcnn)
      MTCNNbox = list(zip(MTCNNrect,Confidence))

    text = calculate_iou(gtbox,MTCNNbox)
    
    f.write(text)
    counter = counter + num_faces + 2

  try:
    print(no_counter)
  except: 
    pass
  print(fold_num,"is done")
  f.close()

01 is done
02 is done
03 is done
04 is done
05 is done
06 is done
07 is done
08 is done
09 is done
10 is done
